In [1]:
# Importing libraries
import pandas as pd
import cdsapi
import xarray as xr
import numpy as np
import time

In [2]:
# Importing coordinates
local = pd.read_csv(r"C:\Users\joaoa\Desktop\Doutoramento\Download ERA5\local.cvs")
local

,point,latitude,longitude
0,0,42.2,-8.2
1,1,42.1,-8.6
2,2,42.1,-8.5
3,3,42.1,-8.4
4,4,42.1,-8.3
...,...,...,...
1007,1058,37.1,-7.8
1008,1059,37.1,-7.7
1009,1062,37.0,-8.0
1010,1063,37.0,-7.9


In [3]:
# Creating columns
col_list = ["ti", "tf"]
yi = 1950 #1950

for row in range(0, len(local)):
    col_list.append('P' + (4-len(str(local.iloc[row,0])))*'0' + str(local.iloc[row,0]))

    
# Cleaning data and filtering dates
Y = range(yi, 2024) #yi-2024
P = range(0, 1012) #0-1012
df = pd.DataFrame()

for year in Y: 
    i = time.time()
    y1 = df
    
    ds = xr.open_dataset(r"D:\Precipitation\by_civil_year\Ficheiros NC\era5land_" + str(year) + ".nc")
    df = ds.to_dataframe()
    df = df.reset_index()
    df["longitude"] = round(df["longitude"], 1)
    df["latitude"] = round(df["latitude"], 1)
    df["point"] = -1
    
    for row in range(0, len(local)):
        df.loc[(df["latitude"] == local.loc[row, "latitude"]) & (df["longitude"] == local.loc[row, "longitude"]), "point"] = local.loc[row, "point"]

    df = df[df["point"] >= 0]
    df = df.sort_values(by=["point", "time"])
    df = df.rename(columns={"tp": "precipitation"})
    df = df[["point", "latitude", "longitude", "time", "precipitation"]]
    df = df.reset_index(drop = True)
    y2 = df
    
    # Creating and saving water years
    if year > yi:
        
        df_wy = pd.DataFrame(index = range(0, 8760), columns = col_list)
        
        # Filling ti
        df_wy.iloc[:2208, 0] = y1.iloc[-2208:, 3]
        df_wy.iloc[-6552:-5136, 0] = y2.iloc[:1416, 3]
        df_wy.iloc[-5136:, 0] = y2.iloc[-7344:-2208, 3]
        
        # Filling tf
        df_wy.iloc[:2207, 1] = y1.iloc[-2207:, 3]
        df_wy.iloc[-6553:-5136, 1] = y2.iloc[:1417, 3]
        df_wy.iloc[-5136:, 1] = y2.iloc[-7343:-2207, 3]
        
        # Filling precipitation
        for point in P:
            df_wy.iloc[:2207, point+2] = y1.loc[y1["point"] == local.iloc[point, 0]].iloc[-2207:, 4]
            df_wy.iloc[-6553:-5136, point+2] = y2.loc[y2["point"] == local.iloc[point, 0]].iloc[:1417, 4]
            df_wy.iloc[-5136:, point+2] = y2.loc[y2["point"] == local.iloc[point, 0]].iloc[-7343:-2207, 4]
            
        # Converting m to mm
        for row in range(8759, -1, -1):
            if df_wy.iloc[row, 1].hour == 1:
                df_wy.iloc[row, 2:] = df_wy.iloc[row, 2:]*1000
            else:
                df_wy.iloc[row, 2:] = (df_wy.iloc[row, 2:] - df_wy.iloc[row-1, 2:])*1000
        
        # Saving dataframe to csv
        print("Water year " + str(year-1) + "_" + str(year) + " downloaded. Nulls: " + str(df_wy.isnull().sum().sum()) + ". Min: " + str(df_wy.iloc[:, 2:].min().min()) + ". Max: " + str(df_wy.iloc[:, 2:].max().max()) + ". Time: " + str(int(time.time()-i)) + " seconds.")
        df_wy[df_wy.iloc[:, 2:] < 0] = 0
        df_wy.to_csv(r"D:\Precipitation\by_water_year\era5land_total_precipitation_" + str(year-1) + "_" + str(year) + "_(mm).csv", compression = "zip", index = False)

Water year 1950_1951 downloaded. Nulls: 0. Min: -0.001434236764907837. Max: 9.981362149119377. Time: 115 seconds.
Water year 1951_1952 downloaded. Nulls: 0. Min: -0.001434236764907837. Max: 11.529259383678436. Time: 117 seconds.
Water year 1952_1953 downloaded. Nulls: 0. Min: -0.0013373792171478271. Max: 13.822834938764572. Time: 107 seconds.
Water year 1953_1954 downloaded. Nulls: 0. Min: -0.0012330710887908936. Max: 12.07558810710907. Time: 119 seconds.
Water year 1954_1955 downloaded. Nulls: 0. Min: -0.0018104910850524902. Max: 8.704138919711113. Time: 108 seconds.
Water year 1955_1956 downloaded. Nulls: 0. Min: -0.0018104910850524902. Max: 15.270758420228958. Time: 108 seconds.
Water year 1956_1957 downloaded. Nulls: 0. Min: -0.0016354024410247803. Max: 9.926239028573036. Time: 107 seconds.
Water year 1957_1958 downloaded. Nulls: 0. Min: -0.0017955899238586426. Max: 9.21967625617981. Time: 108 seconds.
Water year 1958_1959 downloaded. Nulls: 0. Min: -0.0018738210201263428. Max: 17.

Water year 2021_2022 downloaded. Nulls: 0. Min: -0.0015534460544586182. Max: 12.993276119232178. Time: 236 seconds.
Water year 2022_2023 downloaded. Nulls: 0. Min: -0.0015981495380401611. Max: 14.610011130571365. Time: 206 seconds.


In [4]:
# 01 Out 01:00:00 até 31 Dez 23:00:00 [-2207:] y1 - 2207
# 01 Jan 00:00:00 até 29 Fev 00:00:00 [:1417] y2 - 1417
# 01 Mar 01:00:00 até 01 Out 00:00:00 [-7343:-2207] y2 - 5136